Chapter 20 Graphs<br>

A graph is a pair (V, E) where V is any set and E is a set of
pairs of elements of V . We call V the vertex set and E the edge set.

Example:
G = ({1,2,3,4}, {(1,2), (1,3), (1,4)})<br>

The graph G has 4 vertices and 3 edges. Such a graph is often depicted
4 labeled circles for vertices and lines between them depicting the edges.
Because tuples are ordered, we would add arrowheads. This indicates a
directed graph or digraph. If the ordering of the vertices in an edge
does not matter, we have an undirected graph. There will be no major
difference between an undirected graph and a digraph that has a matched
pair (v, u) for every (u, v) in E.<br>

Usually, we will disallow self-loops, edges that start and end at the
same vertex, and multiple edges between the same pair of vertices. Graphs
without self-loops and multiple edges are called simple graphs.<br>

If two vertices are connected by an edge, we say they are adjacent. We
all call such vertices neighbors. For an edge e = (u, v), we say that the
vertices u and v are incident to the edge e. The degree of a vertex
is the number of neighbors it has. For digraphs, we distinguish between
in-degree and out-degree, the number of in-neighbors and out-neighbors
respectively.

20.1 A Graph ADT<br>

- __ init __(V, E) : Initialize a new graph with vertex set V and edge set E.
- vertices() : Return an iterable collection of the vertices.
- edges() : Return an iterable collection of the edges.
- addvertex(v) : Add a new vertex to the graph. The new vertex is identified with the v object.
- addedge(u, v) : Add a new edge to the graph between the vertices with keys u and v.
- removeedge(u,v) : Remove the edge u,v from the graph.
- __ contains __ (v) : Return True if the vertex v is in the graph and return False otherwise.
- hasedge(u,v) : Return True if the edge (u,v) is in the graph and return False otherwise.
- nbrs(v) : Return an iterable collection of the (out)neighbors of v, i.e. those vertices w such that (v, w) is an edge. (For directed graphs, this is the collection of out-neighbors.)
- __ len __ () : Return the number of vertices in the graph.

20.2 The EdgeSetGraph Implementation

In [1]:
class EdgeSetGraph:
    def __init__(self, V = (), E = ()):
        self._V = set()
        self._E = set()
        for v in V: self.addvertex(v)
        for u, v in E: self.addegde(u, v)

    def vertices(self):
        return iter(self._V)
    
    def edges(self):
        return iter(self._E)
    
    def addvertex(self, v):
        self._V.add(v)

    def addedge(self, u, v):
        self._E.add((u, v))

    def removeedge(self, u, v):
        self._E.remove((u, v))

    def __contains__(self, v):
        return v in self._V
    
    def hasedge(self, u, v):
        return (u, v) in self._E
    
    def nbrs(self, v):
        return (w for u, w in self._E if u == v)
    
    def __len__(self):
        return len(self._V)

To make an undirected version of this class, we can will replace the
tuples we are currently using with sets. The problem here is that Python
doesn’t let us use sets as elements of sets. Remember that mutable types
like sets and lists cannot be used this way. Thankfully, Python provides an
immutable set type called frozenset. It is just like a set except that it
cannot be changed. It can be used for our edge set in the undirected graph
as follows.

In [2]:
class UndirectedEdgeSetGraph(EdgeSetGraph):
    def addedge(self, u, v):
        self._E.add(frozenset({u, v}))

    def removeedge(self, u, v):
        self._E.remove(frozenset({u, v}))

    def nbrs(self, v):
        for u , w in self._E:
            if u == v:
                yield w
            elif w == v:
                yield u

20.3 The AdjacencySetGraph Implementation

In [16]:
class AdjacencySetGraph:
    def __init__(self, V = (), E = ()):
        self._V = set()
        self._nbrs = {}
        for v in V: self.addvertex(v)
        for e in E: self.addedge(*e)

    def vertices(self):
        return iter(self._V)
    
    def edges(self):
        for u in self._V:
            for v in self._nbrs(u):
                yield (u, v)
    
    def addvertex(self, v):
        self._V.add(v)
        self._nbrs[v] = set()

    def addedge(self, u, v):
        self._nbrs[u].add(v)

    def removeedge(self, u, v):
        self._nbrs[u].remove(v)

    def __contains__(self, v):
        return v in self._nbrs
    
    def nbrs(self, v):
        return iter(self._nbrs[v])
    
    def __len__(self):
        return len(self._nbrs)
    
    def hasedge(self, u, v):
        return v in self._nbrs[u]
    
    # def connected(self, a, b):
    #     if a == b: return True
    #     return any(self.connected(nbr, b) for nbr in self.nbrs(a))

    def connected(self, a, b):
        return self._connected(a, b, set())

    def _connected(self, a, b, visited):
        if a in visited: return False
        if a == b: return True
        visited.add(a)
        return any(self._connected(nbr, b, visited) for nbr in self.nbrs(a))
    
G = AdjacencySetGraph({1, 2, 3}, {(1, 2), (2, 1), (1, 3)})
print("neighbors of 1:", list(G.nbrs(1)))
print("neighbors of 2:", list(G.nbrs(2)))
print("neighbors of 3:", list(G.nbrs(3)))

neighbors of 1: [2, 3]
neighbors of 2: [1]
neighbors of 3: []


If self. nbrs[u] in hasedge method were a list, the method could take time linear in the degree of u.<br>

To make an undirected version of the AdjacencySetGraph, we will let
it behave like a directed graph in which every edge has a twin in the oppo-
site direction. This affects our addedge and removeedge methods. It also
requires us to be a little more careful about the edges method so as not
to return twice as many edges as before. We will use a set of frozensets as
before to eliminate duplicates.

In [4]:
from ds2.graph import AdjacencySetGraph

class UndirectedAdjacencySetGraph(AdjacencySetGraph):
    def addedge(self, u, v):
        AdjacencySetGraph.addedge(self, u, v)
        AdjacencySetGraph.addedge(self, v, u)

    def removeedge(self, u, v):
        AdjacencySetGraph.removeedge(self, u, v)
        AdjacencySetGraph.removeedge(self, v, u)

    def edges(self):
        E = {frozenset(e) for e in AdjacencySetGraph.edges(self)}
        return iter(E)

20.4 Paths and Connectivity<br>

A path in a graph G = (V, E) is a sequence of vertices connected by edges.
That is, a nonempty sequence of vertices (v0 , v1 , . . . , vk ) is a path from v0
to vk as long as (vi−1 , vi ) ∈ E for all i ∈ 1, . . . , k. We say a **path is simple**
if it does not repeat any vertices. The length of a path is the number of
edges. A single vertex can be seen as a path of length zero.<br>

A cycle is a path of length at least one that starts and ends at the same
vertex. The length of a cycle is the number of edges. A cycle is simple
if is is a cycle and removing the last edge results in a simple path, i.e., there
are no repeated vertices until the last one.
To solidify these definitions, we could write a couple methods to check
them.

In [5]:
def ispath(self, V):
    """Return True if and only if the vertices V form a path."""
    return V and all(self.hasedge(V[i-1], V[i]) for i in range(1, len(V)))

def issimplepath(self, V):
    """Return True if and only if the vertices V form a simple path."""
    return self.ispath(V) and len(V) == len(set(V))

def iscycle(self, V):
    """Return True if and only if the vertices V form a cycle."""
    return self.ispath(V) and V[0] == V[-1]

def issimplecycle(self, V):
    """Return True if and only if the vertices V form a simple cycle."""
    return self.iscycle(V) and self.issimplepath(V[:-1])

In [6]:
G = AdjacencySetGraph({1,2,3,4}, {(1,2),(3,1), (2,3), (3,4), (4,3)})
print("[1,2,3,1] is a path", G.ispath([1,2,3,1]))
print("[1,2,3,1] is a simple path", G.issimplepath([1,2,3,1]))
print("[1,2,3] is a simple path", G.issimplepath([1,2,3]))
print("[1,2,3] is a simple cycle:", G.issimplecycle([1,2,3]))
print("[1,2,3,1] is a simple cycle:", G.issimplecycle([1,2,3]))
print("[1,2,3,4] is a simple path:", G.issimplepath([1,2,3,4]))
print("[1,2,3,4] is a simple cycle:", G.issimplecycle([1,2,3,4]))
print("[1,2,3,4,3,1] is a cycle:", G.iscycle([1,2,3,4,3,1]))
print("[1,2,3,4,3,1] is a simple cycle:", G.issimplecycle([1,2,3,4,3,1]))

[1,2,3,1] is a path True
[1,2,3,1] is a simple path False
[1,2,3] is a simple path True
[1,2,3] is a simple cycle: False
[1,2,3,1] is a simple cycle: False
[1,2,3,4] is a simple path: True
[1,2,3,4] is a simple cycle: False
[1,2,3,4,3,1] is a cycle: True
[1,2,3,4,3,1] is a simple cycle: False


We say that u is connected to v if there exists a path that starts at u
and ends at v. For an undirected graph, if u is connected to v, then v is
connected to u. In such graphs, we can partition the vertices into subsets
called connected components that are all pairwise connected.
For a directed graph, two vertices u and v are strongly connected if
u is connected to v and also v is connected to u.

In [7]:
def connected(self, a, b):
    if a == b: return True
    return any(self.connected(nbr, b) for nbr in self.nbrs(a))

In [15]:
G = AdjacencySetGraph({1,2,3}, {(1,2), (2,3)})
assert(G.connected(1,2))
assert(G.connected(1,3))
assert(not G.connected(3,1))
print("First graph is okay.")
H = AdjacencySetGraph({1,2,3}, {(1,2), (2,1), (2,3)})
try:
    H.connected(1,3)
except RecursionError:
    print("There was too much recursion!")

First graph is okay.
There was too much recursion!


It’s clear that if the graph has any cycles, we can’t check connectivity
this way. To deal with cycles, we can keep a set of visited vertices. Recall
that this is called memoization.

In [9]:
def connected(self, a, b):
    return self._connected(a, b, set())

def _connected(self, a, b, visited):
    if a in visited: return False
    if a == b: return True
    visited.add(a)
    return any(self._connected(nbr, b, visited) for nbr in self.nbrs(a))

In [17]:
H = AdjacencySetGraph({1,2,3}, {(1,2), (2,1), (2,3)})
try:
    assert(H.connected(1,2))
    assert(H.connected(1,3))
except RecursionError:
    print("There was too much recursion!")
print("It works now!")

It works now!


Visualize and understand graphs better [here](https://csacademy.com/app/graph_editor/)